In [1]:
%reset -f

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier, StackingClassifier
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, optimizers
import os

## Загружаем данные

In [4]:
folder_path = r"selected_data.csv" # Путь к папке с CSV файлами

dataset = pd.read_csv(folder_path, index_col=0)
print(dataset.shape)
dataset.head()

(70000, 4)


,FrL,FrR,OcR,target
0,0.076875,0.151062,0.049438,0
1,0.050438,0.133375,0.028438,0
2,0.031438,0.122813,0.017063,0
3,0.031750,0.109500,0.007313,0
4,0.029750,0.106625,-0.036812,0


In [6]:
splitline = int(dataset.shape[0]*0.85)

X = dataset.drop(columns=['target'])
y = dataset.target

X_train, X_test = X[:splitline], X[splitline:]
y_train, y_test = y[:splitline], y[splitline:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((59500, 3), (10500, 3), (59500,), (10500,))

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Models

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, SimpleRNN, Dropout, Input
from sklearn.metrics import accuracy_score

In [13]:
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    
    # Сбрасываем индексы в y, чтобы они совпадали с индексами X
    y = y.reset_index(drop=True)
    for i in range(len(X) - seq_length + 1):
        X_seq.append(X[i:i + seq_length])
        y_seq.append(y[i + seq_length - 1])
        
    return np.array(X_seq), np.array(y_seq)

In [15]:
# Параметры
seq_length = 7
num_features = X_train.shape[1]

# Преобразуем тренировочные и тестовые данные
X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, seq_length)

In [17]:
model = Sequential([
    Input(shape=(seq_length, num_features)),
    Bidirectional(layers.LSTM(512, return_sequences=False)),
    Dense(256, activation='relu'),
    Dense(3, activation='softmax')
])

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_test_seq, y_test_seq),
    epochs=40,
    batch_size=32)

Epoch 1/40
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 64s 33ms/step - accuracy: 0.5152 - loss: 0.9824 - val_accuracy: 0.4875 - val_loss: 0.9138
Epoch 2/40
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 61s 33ms/step - accuracy: 0.5647 - loss: 0.9090 - val_accuracy: 0.5051 - val_loss: 0.9019
Epoch 3/40
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 61s 33ms/step - accuracy: 0.5768 - loss: 0.8863 - val_accuracy: 0.5237 - val_loss: 0.8863
Epoch 4/40
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 62s 33ms/step - accuracy: 0.5941 - loss: 0.8627 - val_accuracy: 0.4922 - val_loss: 0.9292
Epoch 5/40
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 61s 33ms/step - accuracy: 0.6006 - loss: 0.8468 - val_accuracy: 0.5842 - val_loss: 0.8525
Epoch 6/40
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 61s 33ms/step - accuracy: 0.6083 - loss: 0.8295 - val_accuracy: 0.5681 - val_loss: 0.8822
Epoch 7/40
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 61s 33ms/step - accuracy: 0.6175 - loss: 0.8155 - val_accuracy: 0.5479 - val_loss: 0.8960
Epoch 8/40
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 61s 33ms/step - accuracy: 0.6190 -

In [22]:
model.save('BiLSTM_512-256_model.h5')